In [9]:
import numpy as np
import pandas as pd
import sklearn
import xgboost
import lightgbm
import time
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.utils import all_estimators
from sklearn.base import ClassifierMixin

In [2]:
removed_classifiers = [
    "ClassifierChain",
    "ComplementNB",
    "GradientBoostingClassifier",
    "GaussianProcessClassifier",
    "HistGradientBoostingClassifier",
    "MLPClassifier",
    "LogisticRegressionCV", 
    "MultiOutputClassifier", 
    "MultinomialNB", 
    "OneVsOneClassifier",
    "OneVsRestClassifier",
    "OutputCodeClassifier",
    "RadiusNeighborsClassifier",
    "VotingClassifier",
    "LabelPropagation",
    "LabelSpreading",
    'SVC',
]

CLASSIFIERS = [
    est
    for est in all_estimators()
    if (issubclass(est[1], ClassifierMixin) and (est[0] not in removed_classifiers))
]

CLASSIFIERS.append(("XGBClassifier", xgboost.XGBClassifier))
CLASSIFIERS.append(("LGBMClassifier", lightgbm.LGBMClassifier))

In [3]:
# Read in the data
df = pd.read_csv('landcovernet_africa_data.csv')

In [4]:
# separate the data from the labels
# f1 (costal) and f2 (blue) are bands for landsat-8
X, Y = df[['f1', 'f2']], df.labels

In [5]:
# 80/20 split for the train/test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=1234)

In [22]:
# fit models using LazyPredict
start_time = time.time()
clf = LazyClassifier(classifiers = CLASSIFIERS, predictions=True)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)
print("--- %s seconds ---" % (time.time() - start_time))

  0%|          | 0/26 [00:00<?, ?it/s]

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


100%|██████████| 26/26 [00:27<00:00,  1.06s/it]

--- 27.45751118659973 seconds ---


In [13]:
print(models)

                               Accuracy  Balanced Accuracy ROC AUC  F1 Score  \
Model                                                                          
DecisionTreeClassifier             0.96               0.89    None      0.96   
ExtraTreeClassifier                0.96               0.89    None      0.96   
ExtraTreesClassifier               0.96               0.89    None      0.96   
BaggingClassifier                  0.96               0.87    None      0.96   
RandomForestClassifier             0.96               0.87    None      0.96   
KNeighborsClassifier               0.95               0.86    None      0.95   
LGBMClassifier                     0.72               0.42    None      0.70   
NearestCentroid                    0.48               0.38    None      0.51   
GaussianNB                         0.58               0.36    None      0.57   
PassiveAggressiveClassifier        0.61               0.29    None      0.58   
QuadraticDiscriminantAnalysis      0.63 

In [ ]:
# Select Model
# Filter for only a certain accuracy level
top5_models = models.take([0,1,2,3,4,5])
# Select and store the fastest model from the above filter tabled
selected_model = top5_models[(top5_models["Time Taken"] == min(top5_models["Time Taken"]))].index[0]

print(selected_model)

The model which we shall use moving forward will be the ExtraTreeClassifier.